# Better Way 30. 속성을 리팩토링하는 대신 @property를 고려하자

- BW 29에서 내장 @property 데코레이터를 사용하여 인스턴스의 속성에 접근할 수 있었음.
- @property를 단순 숫자 속성을 즉석에서 계산하는 방식으로 변경하여, 호출부를 변경하지 않아도 새로운 동작을 하게 할 수 있음.
- 참고: [데코레이터 설명 및 사용법](http://schoolofweb.net/blog/posts/파이썬-데코레이터-decorator)

In [4]:
from datetime import datetime, timedelta

class Bucket(object):
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period) # 참고: datetime 객체를 비교할 떄, timedelta 객체가 반환됨
        self.reset_time = datetime.now()
        self.quota = 0  
        
    def __repr__(self):
        return 'Bucket(quota=%d)' % self.quota


In [5]:
def fill(bucket, amount):
    now = datetime.now()
    if now - bucket.reset_time > bucket.period_delta:
        bucket.quota = 0
        bucket.reset_time = now
    bucket.quota += amount

def deduct(bucket, amount):
    now = datetime.now()
    if now - bucket.reset_time > bucket.period_delta:
        return False
    if bucket.quota - amount < 0:
        return False
    bucket.quota -= amount
    return True

In [6]:
bucket = Bucket(60)
fill(bucket, 100)
print(bucket)

Bucket(quota=100)


In [7]:
if deduct(bucket, 99):
    print('Had 99 quota')
else:
    print('Not enough for 99 quota')
print(bucket)

Had 99 quota
Bucket(quota=1)


In [8]:
if deduct(bucket, 3):
    print('Had 3 quota')
else:
    print('Not enouch for 3 quota')
print(bucket)

Not enouch for 3 quota
Bucket(quota=1)


위 구현의 문제점
- 양동이 할당량이 어떤 수준에서 시작하는지 모른다는 점이다.
- 즉, deduct 함수 결과가 False일 때, Bucket의 할당량이 소진되어서인지 아니면 처음부터 Bucket에 할당량이 없어서인지 알 수가 없다.
- 해결: 클래스에 max_quota와 quota_consumed 변경을 추적하도록 수정한다.

In [9]:
class Bucket(object):
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.max_quota = 0
        self.quota_consumed = 0
        
    def __repr__(self):
        return 'Bucket(max_quota=%d, quota_consumed=%d)' % (self.max_quota, self.quota_consumed)
    
    @property
    def quota(self):
        return self.max_quota - self.quota_consumed
    
    @quota.setter
    def quota(self, amount):
        delta = self.max_quota - amount
        if amount == 0:
            # 새 기간의 할당량을 리셋함
            self.quota_consumed = 0
            self.max_quota = 0
        elif delta < 0:
            # 새 기간의 할당량을 채움
            assert self.quota_consumed == 0
            self.max_quota = amount
        else:
            # 기간 동안 할당량을 소비함
            assert self.max_quota >= self.quota_consumed
            self.quota_consumed += delta

In [10]:
bucket = Bucket(60)
print('Initial', bucket)
fill(bucket, 100)
print('Filled', bucket)

if deduct(bucket, 99):
    print('Had 99 quota')
else:
    print('Not enough for 99 quota')
    
print('Now', bucket)

if deduct(bucket, 3):
    print('Had 3 quota')
else:
    print('Not enough for 3 quota')

print('Still', bucket)

Initial Bucket(max_quota=0, quota_consumed=0)
Filled Bucket(max_quota=100, quota_consumed=0)
Had 99 quota
Now Bucket(max_quota=100, quota_consumed=99)
Not enough for 3 quota
Still Bucket(max_quota=100, quota_consumed=99)


핵심 정리
- 기존의 인스턴스 속성에 새 기능을 부여하려면 @property를 사용하자.
- @property를 사용하여 점점 나은 데이터 모델로 발전시키자.
- @property를 너무 많이 사용한다면 클래스와 이를 호출하는 모든 곳을 리팩토링하는 방안을 고려하자.